# Basic Simulation Usage

In [1]:
import sys
sys.path.append('../../../../lib/')
import libry as ry
import numpy as np
import time
import matplotlib.pyplot as plt

### Generate Configuration and Simulation

In [2]:
C = ry.Config()
C.addFile("pandasTable.g")
obj = C.addFrame("object", args="mass: 0.1")
obj.setPosition([.2,.2,1.0])
obj.setShape(ry.ST.ssBox, [.05, .05, .05, 0.001])
obj.setColor([1,0,1])
q_home = C.getJointState()
S = ry.Simulation(C, ry.SimulatorEngine.physx, 101)

tau = .01

In [6]:
# Simulate without any controls
for _ in range(100):
    S.step([], 0.01, ry.ControlMode.none)

In [3]:
# align endeffector with object
gripper_pos = 0.0
for t in range(500):
    time.sleep(tau)
    
    q = S.get_q()
    C.setJointState(q) # set your robot model to match the real q
    [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])
    [y2,J2] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "object"])
    [y3, J3] = C.evalFeature(ry.FS.qItself, [])
    y3 = q_home - q
    y *= -1
    y3[:-1] *= 1e-6
    J3[:-1] *= 1e-6
    y3[-1] = gripper_pos - q[-1]
    y3[-1] *=100
    y2 = [1.] - y2
    y = np.concatenate([y*10, y2*10, y3])
    J = np.concatenate([J, J2, J3])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

In [4]:
# close gripper
gripper_pos = -0.04
for t in range(500):
    time.sleep(tau)
    
    q = S.get_q()
    C.setJointState(q) # set your robot model to match the real q
    [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])
    [y2,J2] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "object"])
    [y3, J3] = C.evalFeature(ry.FS.qItself, [])
    y3 = q_home - q
    y *= -1
    y3[:-1] *= 1e-6
    y3[-1] = gripper_pos - y3[-1]
    y3[-1] *= 100
    y2 = [1.] - y2
    y = np.concatenate([y*10, y2*10, y3])
    J = np.concatenate([J, J2, J3])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

In [5]:
# pick up object
gripper_pos = -0.04
for t in range(500):
    time.sleep(tau)
    
    q = S.get_q()
    C.setJointState(q) # set your robot model to match the real q
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripperCenter"])
    [y3, J3] = C.evalFeature(ry.FS.qItself, [])
    y3 = q_home - q
    y = [.2, 0, 1.4] - y
    y3[:-1] *= 1e-6
    y3[-1] = gripper_pos - y3[-1]
    y3[-1] *= 100
    y2 = [1.] - y2
    y = np.concatenate([10*y, y3])
    J = np.concatenate([J, J3])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-6*np.eye(y.shape[0])) @ (y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

In [43]:
# drop object
gripper_pos = 0.5
for t in range(500):
    time.sleep(tau)
    
    q = S.get_q()
    C.setJointState(q) # set your robot model to match the real q
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripperCenter"])
    [y3, J3] = C.evalFeature(ry.FS.qItself, [])
    y3 = q_home - q
    y = [.2, 0, 1.4] - y
    y3[:-1] *= 1e-6
    y3[-1] = gripper_pos - y3[-1]
    y3[-1] *= 100
    y2 = [1.] - y2
    y = np.concatenate([10*y, y3])
    J = np.concatenate([J, J3])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-6*np.eye(y.shape[0])) @ (y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)